# Project - Wrangle and Analyze Data |  "WeRateDogs" Twitter data
## Data Wrangling

In [1]:
import requests
import shutil

import pandas as pd
# To print entire frames:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)


import numpy as np

import json
#import tweepy

import re

#For Plot
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from IPython.display import Image

# Gathering Data

### Reading "twitter-archive-enhanced.csv"

In [128]:
df_twitter = pd.read_csv('twitter-archive-enhanced.csv')

### Programmatically downloading "image_predictions.tsv" from given URL

In [4]:
def file_download(url):
    """ Funtion to download file
    :param url: URL for file location
    :return: return file
    """

    local_filename = url.split('/')[-1]
    r = requests.get(url, stream=True)
    with open(local_filename, 'wb') as f:
        shutil.copyfileobj(r.raw, f)

    return local_filename

# URL showing location of file
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
file_download(url)



'image-predictions.tsv'

In [11]:
# Reading "image_predictions.tsv"
df_image = pd.read_csv('image-predictions.tsv', sep='\t')

### Querying Twitter Data using Tweepy  to query  Twitter API for data included in the "WeRateDogs" Twitter Archive

In [ ]:
# Querying Twitter Data using Tweepy  to query  Twitter API for data  and saving to "tweet_json.txt"

df_twitter_noDuplicateId =  df_twitter.drop_duplicates(subset=['tweet_id'], keep=False)

consumer_key = ' '
consumer_secret = ' '

access_token = ' '
access_token_secret = ' '

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)


tweed_id = df_twitter_noDuplicateId.tweet_id

tweet_json_list = []
for id in tweed_id:
    try:
        tweet = api.get_status(id, tweet_mode='extended')
        tweet_json_list.append(tweet)
        print("Success for tweet ID {}.".format(id))
        time.sleep(5)
    except tweepy.TweepError as e:
        print("Error {} for tweet ID {}.".format(e, id))
        time.sleep(15)


with open('tweet_json.txt', 'w') as outfile:
    for tweet in tweet_json_list:
        json.dump(tweet._json, outfile)
        outfile.write('\n')
  

In [ ]:
# Reading "tweet_json.txt"

final_list = []

with open("tweet_json.txt", 'r') as f:
    count = 0

    line = f.readline() # Reads one line at a time

    while line: # loop through entire file

        json_format = json.loads(line) # parse jason data

        final_list.append({'tweet_id': json_format['id'],
                           'retweet_count': json_format['retweet_count'],
                           'favorite_count': json_format['favorite_count']
                           })
        line = f.readline()



In [ ]:
# Saving to CSV file "tweet_json_text.csv"

df = pd.DataFrame(final_list)

# Setting column order
df = df[['tweet_id','retweet_count','favorite_count']]

# Saving to CSV
df.to_csv("tweet_json_text.csv", sep='\t', encoding = 'utf-8', index=False)



In [12]:
# Reading CSV "tweet_json_text.csv"
df_tweet_viaAPI = pd.read_csv("tweet_json_text.csv", sep="\t")

# Assessing Data

### Assessing Data "twitter-archive-enhanced.csv"

In [16]:
df_twitter.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None


In [7]:
df_twitter.tail(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2354,666029285002620928,NaN,NaN,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a western brown Mitsubishi terrier. Upset about leaf. Actually 2 dogs here. 7/10 would walk the shit out of https://t.co/r7mOb2m0UI,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666029285002620928/photo/1,7,10,a,None,None,None,None
2355,666020888022790149,NaN,NaN,2015-11-15 22:32:08 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here we have a Japanese Irish Setter. Lost eye in Vietnam (?). Big fan of relaxing on stair. 8/10 would pet https://t.co/BLDqew2Ijj,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666020888022790149/photo/1,8,10,None,None,None,None,None


In [10]:
df_twitter.sample(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
303,836397794269200385,NaN,NaN,2017-02-28 02:09:08 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @dog_rates: This is Buddy. He ran into a glass door once. Now he's h*ckin skeptical. 13/10 empowering af (vid by Brittany Gaunt) https:/…,8.178278e+17,4.196984e+09,2017-01-07 20:18:46 +0000,https://twitter.com/dog_rates/status/817827839487737858/video/1,13,10,Buddy,None,None,None,None
1443,696744641916489729,NaN,NaN,2016-02-08 17:17:22 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine - Make a Scene</a>",This is Klevin. He doesn't want his family brainwashed by mainstream media. 10/10 (vid by @AshtonHose) https://t.co/ghhbMAFPW8,NaN,NaN,NaN,https://vine.co/v/i1wrljBUjAu,10,10,Klevin,None,None,None,None


In [17]:
df_twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [18]:
df_twitter[df_twitter['tweet_id'].duplicated()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [18]:
df_twitter.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


#### twitter-archive-enhanced.csv | Assessing dog names

In [15]:
df_twitter['name'].value_counts().sample(10)

Canela    2
Major     1
Rover     1
Chloe     1
Rufio     1
Barney    1
Maude     1
Pablo     2
Rusty     6
Calvin    3
Name: name, dtype: int64

- Documenting above:
    - Missing dog names. Column 'name' has 745 entries have "None"  Value. Names given by dog owner(s) is a very subjective matter, but I did take risk suggesting names like 'a,' 'his,' 'all,' ' such,' 'an.' are the typos.

#### twitter-archive-enhanced.csv | Assessing Column "rating_numerator" and "rating_denominator"

In [17]:
df_twitter['rating_numerator'].unique()

array([  13,   12,   14,    5,   17,   11,   10,  420,  666,    6,   15,
        182,  960,    0,   75,    7,   84,    9,   24,    8,    1,   27,
          3,    4,  165, 1776,  204,   50,   99,   80,   45,   60,   44,
        143,  121,   20,   26,    2,  144,   88], dtype=int64)

In [18]:
df_twitter['rating_numerator'].value_counts(ascending=False).sample(15)

15     2  
44     1  
45     1  
60     1  
84     1  
20     1  
10     461
121    1  
143    1  
4      17 
182    1  
88     1  
75     2  
99     1  
204    1  
Name: rating_numerator, dtype: int64

In [19]:
df_twitter[df_twitter['rating_numerator'] == 1]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
605,798576900688019456,NaN,NaN,2016-11-15 17:22:24 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @dog_rates: Not familiar with this breed. No tail (weird). Only 2 legs. Doesn't bark. Surprisingly quick. Shits eggs. 1/10 https://t.co/…,6.661041e+17,4.196984e+09,2015-11-16 04:02:55 +0000,https://twitter.com/dog_rates/status/666104133288665088/photo/1,1,10,None,None,None,None,None
1446,696490539101908992,6.964887e+17,4.196984e+09,2016-02-08 00:27:39 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",After reading the comments I may have overestimated this pup. Downgraded to a 1/10. Please forgive me,NaN,NaN,NaN,NaN,1,10,None,None,None,None,None
1869,675153376133427200,NaN,NaN,2015-12-11 03:21:23 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",What kind of person sends in a picture without a dog in it? 1/10 just because that's a nice table https://t.co/RDXCfk8hK0,NaN,NaN,NaN,https://twitter.com/dog_rates/status/675153376133427200/photo/1,1,10,None,None,None,None,None
1940,673716320723169284,6.737159e+17,4.196984e+09,2015-12-07 04:11:02 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",The millennials have spoken and we've decided to immediately demote to a 1/10. Thank you,NaN,NaN,NaN,NaN,1,10,None,None,None,None,None
2038,671550332464455680,6.715449e+17,4.196984e+09,2015-12-01 04:44:10 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",After 22 minutes of careful deliberation this dog is being demoted to a 1/10. The longer you look at him the more terrifying he becomes,NaN,NaN,NaN,NaN,1,10,None,None,None,None,None
2091,670783437142401025,NaN,NaN,2015-11-29 01:56:48 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Flamboyant pup here. Probably poisonous. Won't eat kibble. Doesn't bark. Slow af. Petting doesn't look fun. 1/10 https://t.co/jxukeh2BeO,NaN,NaN,NaN,https://twitter.com/dog_rates/status/670783437142401025/photo/1,1,10,None,None,None,None,None
2261,667549055577362432,NaN,NaN,2015-11-20 03:44:31 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Twitter Web Client</a>",Never seen dog like this. Breathes heavy. Tilts head in a pattern. No bark. Shitty at fetch. Not even cordless. 1/10 https://t.co/i9iSGNn3fx,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667549055577362432/photo/1,1,10,None,None,None,None,None
2335,666287406224695296,NaN,NaN,2015-11-16 16:11:11 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is an Albanian 3 1/2 legged Episcopalian. Loves well-polished hardwood flooring. Penis on the collar. 9/10 https://t.co/d9NcXFKwLv,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666287406224695296/photo/1,1,2,an,None,None,None,None
2338,666104133288665088,NaN,NaN,2015-11-16 04:02:55 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Not familiar with this breed. No tail (weird). Only 2 legs. Doesn't bark. Surprisingly quick. Shits eggs. 1/10 https://t.co/Asgdc6kuLX,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666104133288665088/photo/1,1,10,None,None,None,None,None


In [ ]:
# tweet_id = 666287406224695296 , score 9

In [20]:
# choosing rows with df_twitter['rating_numerator'] == 1
twitter_numerator_rating_one = df_twitter[df_twitter['rating_numerator'] == 1]

for item in twitter_numerator_rating_one['text']:
    text_split = re.split('\s+', item) # \s+ split by single space or more
    filter_number = re.findall(r'\d+[/]\d+', item)

    print("{}".format(filter_number))


['1/10']
['1/10']
['1/10']
['1/10']
['1/10']
['1/10']
['1/10']
['1/2', '9/10']
['1/10']


- Documenting above:
    - For 'rating_numerator' = 1,  Erroneous value at  'rating_numerator' for "tweet_id" = 666287406224695296. Column 'text' show rating '9' were as   column 'rating_numerator' is 1.  


In [3]:
df_twitter[df_twitter['rating_numerator'] == 4]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
883,760252756032651264,NaN,NaN,2016-08-01 23:15:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Fido. He can tell the weather. Not good at fetch tho. Never comes when called. 4/10 would probably still pet https://t.co/4gOv2Q3iKP,NaN,NaN,NaN,https://twitter.com/dog_rates/status/760252756032651264/photo/1,4,10,Fido,None,None,None,None
912,757596066325864448,NaN,NaN,2016-07-25 15:19:12 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here's another picture without a dog in it. Idk why you guys keep sending these. 4/10 just because that's a neat rug https://t.co/mOmnL19Wsl,NaN,NaN,NaN,https://twitter.com/dog_rates/status/757596066325864448/photo/1,4,10,None,None,None,None,None
1004,747816857231626240,NaN,NaN,2016-06-28 15:40:07 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Viewer discretion is advised. This is a terrible attack in progress. Not even in water (tragic af). 4/10 bad sherk https://t.co/L3U0j14N5R,NaN,NaN,NaN,https://twitter.com/dog_rates/status/747816857231626240/photo/1,4,10,a,None,None,None,None
1165,722974582966214656,NaN,NaN,2016-04-21 02:25:47 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a,NaN,NaN,NaN,https://twitter.com/dog_rates/status/722974582966214656/photo/1,4,20,None,None,None,None,None
1219,714631576617938945,NaN,NaN,2016-03-29 01:53:39 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Benedict. He's a feisty pup. Needs a brushing. Portable af. Looks very angry actually. 4/10 might not pet https://t.co/3oeFfHjv0Z,NaN,NaN,NaN,https://twitter.com/dog_rates/status/714631576617938945/photo/1,4,10,Benedict,None,None,None,None
1303,707420581654872064,NaN,NaN,2016-03-09 04:19:44 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Keurig. He's a rare dog. Laughs like an idiot tho. Head is basically a weapon. Poorly maintained goatee 4/10 https://t.co/xOrUyj7K30,NaN,NaN,NaN,https://twitter.com/dog_rates/status/707420581654872064/photo/1,4,10,Keurig,None,None,None,None
1459,695064344191721472,NaN,NaN,2016-02-04 02:00:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This may be the greatest video I've ever been sent. 4/10 for Charles the puppy, 13/10 overall. (Vid by @stevenxx_) https://t.co/uaJmNgXR2P",NaN,NaN,NaN,https://twitter.com/dog_rates/status/695064344191721472/video/1,4,10,None,None,None,None,None
1598,686035780142297088,6.860340e+17,4.196984e+09,2016-01-10 04:04:10 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","Yes I do realize a rating of 4/20 would've been fitting. However, it would be unjust to give these cooperative pups that low of a rating",NaN,NaN,NaN,NaN,4,20,None,None,None,None,None
1629,684567543613382656,NaN,NaN,2016-01-06 02:49:55 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Bobby. He doesn't give a damn about personal space. Convinced he called shotgun first. 4/10 not the best dog https://t.co/b8XW69gSaU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/684567543613382656/photo/1,4,10,Bobby,None,None,None,None
1701,680940246314430465,NaN,NaN,2015-12-27 02:36:20 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Alice. She's an idiot. 4/10 https://t.co/VQXdwJfkyS,NaN,NaN,NaN,https://twitter.com/dog_rates/status/680940246314430465/photo/1,4,10,Alice,None,None,None,None


In [22]:
df_twitter[df_twitter['rating_numerator'] == 0]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
315,835152434251116546,NaN,NaN,2017-02-24 15:40:31 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",When you're so blinded by your systematic plagiarism that you forget what day it is. 0/10 https://t.co/YbEJPkg4Ag,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/835152434251116546/photo/1,https://twitter.com/dog_rates/status/835152434251116546/photo/1,https://twitter.com/dog_rates/status/835152434251116546/photo/1",0,10,None,None,None,None,None
1016,746906459439529985,7.468859e+17,4.196984e+09,2016-06-26 03:22:31 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","PUPDATE: can't see any. Even if I could, I couldn't reach them to pet. 0/10 much disappointment https://t.co/c7WXaB2nqX",NaN,NaN,NaN,https://twitter.com/dog_rates/status/746906459439529985/photo/1,0,10,None,None,None,None,None


- Documenting above:
    - Two rows of column 'rating_numerator' has zero value.

        - For "tweet_id" = 835152434251116546, expanded URL does not show the picture of dog thus rated zero. This row can be removed from the dataset.

        - For "tweet_id" = 746906459439529985, expanded URL does show dog, but  "WeRateDogs" rated  zero due to  "plagiarism "  


In [23]:
df_twitter['rating_denominator'].unique()

array([ 10,   0,  15,  70,   7,  11, 150, 170,  20,  50,  90,  80,  40,
       130, 110,  16, 120,   2], dtype=int64)

In [19]:
df_twitter['rating_denominator'].value_counts(ascending=False)

10     2333
11     3   
50     3   
80     2   
20     2   
2      1   
16     1   
40     1   
70     1   
15     1   
90     1   
110    1   
120    1   
130    1   
150    1   
170    1   
7      1   
0      1   
Name: rating_denominator, dtype: int64

- Documenting above:
    - Many rows have 'rating_denominator' not equal to 10


#### twitter-archive-enhanced.csv | Assessing Column "in_reply_to_status_id"

In [20]:
( df_twitter['in_reply_to_status_id'].isnull().sum() /  df_twitter.shape[0] ) * 100  

96.689303904923591

- Documenting above:
    - Column "in_reply_to_status_id" has more than 90% of NaN entries 


#### twitter-archive-enhanced.csv | Assessing Column "in_reply_to_user_id"

In [21]:
( df_twitter['in_reply_to_user_id'].isnull().sum() /  df_twitter.shape[0] ) * 100  

96.689303904923591

- Documenting above:
    - Column "in_reply_to_user_id" has more than 90% of NaN entries 


#### twitter-archive-enhanced.csv | Assessing Column "in_reply_to_status_id"

In [27]:
( df_twitter['retweeted_status_id'].isnull().sum() /  df_twitter.shape[0] ) * 100  

92.317487266553471

- Documenting above:
    - Column "retweeted_status_id" has more than 90% of NaN entries 


#### twitter-archive-enhanced.csv | Assessing Column "retweeted_status_id"

In [28]:
( df_twitter['retweeted_status_user_id'].isnull().sum() /  df_twitter.shape[0] ) * 100  

92.317487266553471

- Documenting above:
    - Column "retweeted_status_user_id" has more than 90% of NaN entries 


#### twitter-archive-enhanced.csv | Assessing Column "retweeted_status_timestamp"

In [29]:
( df_twitter['retweeted_status_timestamp'].isnull().sum() /  df_twitter.shape[0] ) * 100  

92.317487266553471

- Documenting above:
    - Column "retweeted_status_timestamp" has more than 90% of NaN entries 


In [30]:
df_twitter.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None


#### twitter-archive-enhanced.csv | Assessing dog stages

In [24]:
# Finding dog with two stages "dogo" and "floofer"
df_twitter[(df_twitter['doggo'] == 'doggo') & (df_twitter['floofer'] == 'floofer')] 

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
200,854010172552949760,NaN,NaN,2017-04-17 16:34:26 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","At first I thought this was a shy doggo, but it's actually a Rare Canadian Floofer Owl. Amateurs would confuse the two. 11/10 only send dogs https://t.co/TXdT3tmuYk",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/854010172552949760/photo/1,https://twitter.com/dog_rates/status/854010172552949760/photo/1",11,10,None,doggo,floofer,None,None


In [19]:
# Finding dog with two stages "dogo" and "pupper"
df_twitter[(df_twitter['doggo'] == 'doggo') & (df_twitter['pupper'] == 'pupper')]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
460,817777686764523521,NaN,NaN,2017-01-07 16:59:28 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Dido. She's playing the lead role in ""Pupper Stops to Catch Snow Before Resuming Shadow Box with Dried Apple."" 13/10 (IG: didodoggo) https://t.co/m7isZrOBX7",NaN,NaN,NaN,https://twitter.com/dog_rates/status/817777686764523521/video/1,13,10,Dido,doggo,None,pupper,None
531,808106460588765185,NaN,NaN,2016-12-12 00:29:28 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here we have Burke (pupper) and Dexter (doggo). Pupper wants to be exactly like doggo. Both 12/10 would pet at same time https://t.co/ANBpEYHaho,NaN,NaN,NaN,https://twitter.com/dog_rates/status/808106460588765185/photo/1,12,10,None,doggo,None,pupper,None
565,802265048156610565,7.331095e+17,4.196984e+09,2016-11-25 21:37:47 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","Like doggo, like pupper version 2. Both 11/10 https://t.co/9IxWAXFqze",NaN,NaN,NaN,https://twitter.com/dog_rates/status/802265048156610565/photo/1,11,10,None,doggo,None,pupper,None
575,801115127852503040,NaN,NaN,2016-11-22 17:28:25 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Bones. He's being haunted by another doggo of roughly the same size. 12/10 deep breaths pupper everything's fine https://t.co/55Dqe0SJNj,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/801115127852503040/photo/1,https://twitter.com/dog_rates/status/801115127852503040/photo/1",12,10,Bones,doggo,None,pupper,None
705,785639753186217984,NaN,NaN,2016-10-11 00:34:48 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Pinot. He's a sophisticated doggo. You can tell by the hat. Also pointier than your average pupper. Still 10/10 would pet cautiously https://t.co/f2wmLZTPHd,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/785639753186217984/photo/1,https://twitter.com/dog_rates/status/785639753186217984/photo/1",10,10,Pinot,doggo,None,pupper,None
733,781308096455073793,NaN,NaN,2016-09-29 01:42:20 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine - Make a Scene</a>","Pupper butt 1, Doggo 0. Both 12/10 https://t.co/WQvcPEpH2u",NaN,NaN,NaN,https://vine.co/v/5rgu2Law2ut,12,10,None,doggo,None,pupper,None
778,775898661951791106,NaN,NaN,2016-09-14 03:27:11 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","RT @dog_rates: Like father (doggo), like son (pupper). Both 12/10 https://t.co/pG2inLaOda",7.331095e+17,4.196984e+09,2016-05-19 01:38:16 +0000,"https://twitter.com/dog_rates/status/733109485275860992/photo/1,https://twitter.com/dog_rates/status/733109485275860992/photo/1",12,10,None,doggo,None,pupper,None
822,770093767776997377,NaN,NaN,2016-08-29 03:00:36 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @dog_rates: This is just downright precious af. 12/10 for both pupper and doggo https://t.co/o5J479bZUC,7.410673e+17,4.196984e+09,2016-06-10 00:39:48 +0000,"https://twitter.com/dog_rates/status/741067306818797568/photo/1,https://twitter.com/dog_rates/status/741067306818797568/photo/1",12,10,just,doggo,None,pupper,None
889,759793422261743616,NaN,NaN,2016-07-31 16:50:42 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","Meet Maggie &amp; Lila. Maggie is the doggo, Lila is the pupper. They are sisters. Both 12/10 would pet at the same time https://t.co/MYwR4DQKll",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/759793422261743616/photo/1,https://twitter.com/dog_rates/status/759793422261743616/photo/1",12,10,Maggie,doggo,Non

- Documenting above assessment

#### twitter-archive-enhanced.csv | Assessing column timestamp

In [20]:
df_twitter['timestamp'] = pd.to_datetime(df_twitter['timestamp'])

In [27]:
df_twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null datetime64[ns]
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: datetime64[ns](1

- Documenting above:
    - Column 'timestamp' converting from string to datetime

#### twitter-archive-enhanced.csv | Assessing numerator not whole number

In [5]:
# 
df_twitter_decimal_numerator = df_twitter.text.str.extract(r"(\d+\.\d*\/\d+)") 
df_twitter_decimal_numerator = df_twitter_decimal_numerator.dropna()
df_twitter_decimal_numerator 

,0
45,13.5/10
340,9.75/10
695,9.75/10
763,11.27/10
1689,9.5/10
1712,11.26/10


In [9]:
# checking actual values in colum rating_numerator
list_rating_numerator_whole_number_index = []
for item in df_twitter_decimal_numerator.index:
    list_rating_numerator_whole_number_index.append(item)
for index in list_rating_numerator_whole_number_index:
    print(index, df_twitter.loc[index]['rating_numerator'])

45 5
340 75
695 75
763 27
1689 5
1712 26


- Documenting above:
    - Actual rating in Column 'rating_numerator' is different than in column 'text. After decimal,  values are selected instead of the whole number.

#### twitter-archive-enhanced.csv | Separate column for tiny url

In [8]:
df_twitter.text.head(2)

0    This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU                                                     
1    This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV
Name: text, dtype: object

In [7]:
df_twitter.text.str.extract(r"(http[s]:\/\/t\.co\/?[a-zA-Z0-9]{6,})", expand=True)      

,0
0,https://t.co/MgUWQ76dJU
1,https://t.co/0Xxu71qeIV
2,https://t.co/wUnZnhtVJB
3,https://t.co/tD36da7qLQ
4,https://t.co/AtUZn91f7f


- Documenting above:
    - Separate column for tiny url in Column text

In [11]:
df_twitter.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,tiny_url
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None,https://t.co/MgUWQ76dJU
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None,https://t.co/0Xxu71qeIV


### Assessing Data "image_predictions.tsv"

In [16]:
df_image.head(2)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True


In [21]:
df_image.tail(2)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
2073,892177421306343426,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True
2074,892420643555336193,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,1,orange,0.097049,False,bagel,0.085851,False,banana,0.076110,False


In [15]:
df_image.sample(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
588,679111216690831360,https://pbs.twimg.com/ext_tw_video_thumb/679111114081370114/pu/img/hFca8BHjRopgD0cM.jpg,1,kelpie,0.189423,True,beagle,0.121988,True,basset,1.211710e-01,True
475,675149409102012420,https://pbs.twimg.com/media/CV6czeEWEAEdChp.jpg,1,chow,0.999876,True,Tibetan_mastiff,0.000059,True,Tibetan_terrier,2.877850e-05,True
440,674447403907457024,https://pbs.twimg.com/media/CVweVUfW4AACPwI.jpg,1,Brabancon_griffon,0.409909,True,malinois,0.244649,True,bull_mastiff,7.481950e-02,True
1819,834209720923721728,https://pbs.twimg.com/media/C5O1UAaWIAAMBMd.jpg,1,golden_retriever,0.754799,True,Pekinese,0.197861,True,Labrador_retriever,8.654040e-03,True
554,677644091929329666,https://pbs.twimg.com/ext_tw_video_thumb/677644010865999872/pu/img/zVHEMYnJKzq1SauT.jpg,1,Chihuahua,0.626236,True,Italian_greyhound,0.128483,True,swing,5.984040e-02,False
1969,868622495443632128,https://pbs.twimg.com/media/DA33i0XXsAEQtCA.jpg,1,Labrador_retriever,0.868107,True,Great_Pyrenees,0.060973,True,Saint_Bernard,3.348890e-02,True
943,704347321748819968,https://pbs.twimg.com/media/CcZYJniXEAAEJRF.jpg,1,teddy,0.233378,False,feather_boa,0.088474,False,Brittany_spaniel,8.291730e-02,True
1986,872486979161796608,https://pbs.twimg.com/media/DBuyRlTUwAAYhG9.jpg,1,Pembroke,0.931861,True,Cardigan,0.037721,True,Chihuahua,1.196670e-02,True
1843,838561493054533637,https://pbs.twimg.com/media/C6MrOsEXQAENOds.jpg,1,kelpie,0.216562,True,doormat,0.139994,False,dalmatian,1.328200e-01,True
1760,825876512159186944,https://pbs.twimg.com/media/C3YaSnQWAAILgz0.jpg,1,shopping_cart,0.995941,False,shopping_basket,0.004057,False,mousetrap,8.832830e-07,False


In [28]:
df_image['jpg_url'].isnull().sum()

0

In [32]:
df_image.tail()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
2070,891327558926688256,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True
2071,891689557279858688,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
2072,891815181378084864,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
2073,892177421306343426,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True
2074,892420643555336193,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,1,orange,0.097049,False,bagel,0.085851,False,banana,0.076110,False


In [29]:
df_image.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


- Documenting above:
    - Column 'tweet_id' is of type integer. It should be of type string.

In [24]:
df_image.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


### Assessing Data "tweet_json_text.csv"

In [15]:
df_tweet_viaAPI.head(2)

,tweet_id,retweet_count,favorite_count
0,892420643555336193,8443,38416
1,892177421306343426,6220,32908


In [25]:
df_tweet_viaAPI.tail(2)

,tweet_id,retweet_count,favorite_count
2336,666029285002620928,46,129
2337,666020888022790149,511,2557


In [26]:
df_tweet_viaAPI.sample(2)

,tweet_id,retweet_count,favorite_count
890,758355060040593408,1190,3672
1768,677573743309385728,780,2242


In [27]:
df_tweet_viaAPI.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2338 entries, 0 to 2337
Data columns (total 3 columns):
tweet_id          2338 non-null int64
retweet_count     2338 non-null int64
favorite_count    2338 non-null int64
dtypes: int64(3)
memory usage: 54.9 KB


In [28]:
df_tweet_viaAPI.describe()

,tweet_id,retweet_count,favorite_count
count,2.338000e+03,2338.000000,2338.000000
mean,7.422479e+17,2973.071429,8031.337896
std,6.837125e+16,4983.355538,12368.522176
min,6.660209e+17,0.000000,0.000000
25%,6.783361e+17,597.250000,1390.250000
50%,7.187854e+17,1389.500000,3503.000000
75%,7.987010e+17,3468.750000,9874.500000
max,8.924206e+17,84546.000000,164764.000000


#### Quality

##### `Twitter archive enhanced` Table


- Column 'tweet_id' is of type integer. It should be of type string.
- Missing dog name. 745 entries have dog name column  "None" Value.Names given by dog owner(s) is a very subjective matter, but I did take risk suggesting names like  'a,' 'his,' 'all,' ' such,'  'an.'   are the typos.
- Two rows of column 'rating_numerator' has zero value. For 'rating_numerator' = 1,  Erroneous value at  'rating_numerator' for "tweet_id" = 666287406224695296. Column 'text' show rating '9' were as   column 'rating_numerator' is 1.  
- Few observation have column "rating_denominator" is not equal to 10 
- Column "in_reply_to_status_id" has more than 90% of NaN entries  
- Column "in_reply_to_user_id"  has more than 90% of NaN entries  
- Column "retweeted_status_id" has more than 90% of NaN entries  
- Column "retweeted_status_user_id"  has more than 90% of NaN entries  
- Column   "retweeted_status_timestamp"  has more than 90% of NaN entries  
- Column "expanded_urls" has 59 NaN
- Column "timestamp" has "+0000" on every row and is of type String
- Some observations have listed two stage for a single dog.
- Actual rating in Column 'rating_numerator' is different from assigned in  column 'text. After decimal,  values are selected instead of the whole number

#### `Image predictions` Table

- Column 'tweet_id' is of type integer. It should be of type string.

#### `Tweet via  API` Table

- Column 'tweet_id' is of type integer. It should be of type string

#### Tidiness

- Dog stages in separate columns
- 'text' column in the "Twitter archive " table should split  'tint_url' into a separate column
- Merging three dataframes "df_twitter_clean", "df_tweet_viaAPI_clean", and df_image_clean", to "df_master_twitter"

## Clean

#### Making copies of table "df_twitter", "df_image", and "df_tweet_viaAPI"

In [129]:
df_twitter_clean = df_twitter.copy()
df_image_clean = df_image.copy()
df_tweet_viaAPI_clean = df_tweet_viaAPI.copy()

#### Define

##### `Twitter archive enhanced` Table

- Column 'tweet_id' is of type integer. It should be of type string.

#### Code

In [130]:
df_twitter_clean['tweet_id_string'] = df_twitter_clean['tweet_id'].astype(str)

In [54]:
df_twitter_clean.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'doggo', 'floofer', 'pupper', 'puppo',
       'tweet_id_string'],
      dtype='object')

In [131]:
# Rearranging columns
df_twitter_clean = df_twitter_clean[['tweet_id', 'tweet_id_string', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'doggo', 'floofer', 'pupper', 'puppo' ]]

#### Test

In [132]:
df_twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 18 columns):
tweet_id                      2356 non-null int64
tweet_id_string               2356 non-null object
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         23

#### Define

##### `Twitter archive enhanced` Table

- Missing dog name. 745 entries have dog name column  "None" Value.Names given by dog owner(s) is a very subjective matter, but I did take risk suggesting names like  'a,' 'his,' 'all,' ' such,'  'an.'   are the typos.

#### Code

In [133]:
# Filtering "wrong names" to a list
list_wrong_names = list(df_twitter_clean[df_twitter_clean['name'].str.islower()]['name'])

# Replacing wrong names with 'None"
df_twitter_clean['name'] = df_twitter_clean['name'].replace(list_wrong_names, value=None)


#### Test

In [134]:
df_twitter_clean[df_twitter_clean['name'].str.islower()]['name']

Series([], Name: name, dtype: object)

#### Define

##### `Twitter archive enhanced` Table

- Two rows of column 'rating_numerator' has zero value. For 'rating_numerator' = 1,  Erroneous value at  'rating_numerator' for "tweet_id" = 666287406224695296. Column 'text' show rating '9' were as   column 'rating_numerator' is 1.  

#### Code

In [135]:
# Removing rows where 'rating_numerator' == 0
df_twitter_clean = df_twitter_clean[df_twitter_clean.rating_numerator != 0]

In [136]:
df_twitter_clean[df_twitter_clean.tweet_id == 666287406224695296]

,tweet_id,tweet_id_string,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2335,666287406224695296,666287406224695296,NaN,NaN,2015-11-16 16:11:11 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is an Albanian 3 1/2 legged Episcopalian. Loves well-polished hardwood flooring. Penis on the collar. 9/10 https://t.co/d9NcXFKwLv,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666287406224695296/photo/1,1,2,None,None,None,None,None


In [137]:
# changing numerator rating from 1 to 9 per colum 'text' details
df_twitter_clean.loc[df_twitter_clean.tweet_id == 666287406224695296, 'rating_numerator'] = 9

#### Test

In [138]:
df_twitter_clean[df_twitter_clean.rating_numerator == 0]

,tweet_id,tweet_id_string,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [63]:
df_twitter_clean[df_twitter_clean.tweet_id == 666287406224695296]

,tweet_id,tweet_id_string,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2335,666287406224695296,666287406224695296,NaN,NaN,2015-11-16 16:11:11 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is an Albanian 3 1/2 legged Episcopalian. Loves well-polished hardwood flooring. Penis on the collar. 9/10 https://t.co/d9NcXFKwLv,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666287406224695296/photo/1,9,2,None,None,None,None,None


#### Define

##### `Twitter archive enhanced` Table

- Few observation have column "rating_denominator" is not equal to 10 

#### Code

In [139]:
# List of rating_denominator not 10
list_rating_denominator_not_10 = list(df_twitter_clean[df_twitter_clean['rating_denominator'] != 10]['rating_denominator'])

In [140]:
# Replacing List of rating_denominator not 10 by 10
df_twitter_clean['rating_denominator'] = df_twitter_clean['rating_denominator'].replace(list_rating_denominator_not_10, value=10)

#### Test

In [141]:
df_twitter_clean[df_twitter_clean['rating_denominator'] != 10]['rating_denominator']

Series([], Name: rating_denominator, dtype: int64)

#### Define

##### `Twitter archive enhanced` Table

- Column "in_reply_to_status_id" has more than 90% of NaN entries  
- Column "in_reply_to_user_id"  has more than 90% of NaN entries  
- Column "retweeted_status_id" has more than 90% of NaN entries  
- Column "retweeted_status_user_id"  has more than 90% of NaN entries  
- Column   "retweeted_status_timestamp"  has more than 90% of NaN entries  

#### Code

In [143]:
df_twitter_clean.columns

Index(['tweet_id', 'tweet_id_string', 'in_reply_to_status_id',
       'in_reply_to_user_id', 'timestamp', 'source', 'text',
       'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

In [172]:
df_twitter_clean.shape

(2354, 18)

In [173]:
# Counting rows that have non empty 'retweeted_status_id'
df_twitter_clean[pd.notnull(df_twitter_clean.retweeted_status_id)]['retweeted_status_id'].shape

(181,)

In [174]:
# Removing rows that have non empty 'retweeted_status_id' 
df_twitter_clean = df_twitter_clean[pd.isnull(df_twitter_clean.retweeted_status_id)]

In [175]:
df_twitter_clean.shape

(2173, 18)

In [176]:
# Removing columns ""in_reply_to_status_id", "in_reply_to_user_id", "retweeted_status_id", 
# "retweeted_status_user_id", and  "retweeted_status_timestamp"

list_dropColumns = ['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp' ]

df_twitter_clean = df_twitter_clean.drop(list_dropColumns, axis=1)


#### Test

In [177]:
df_twitter_clean.columns

Index(['tweet_id', 'tweet_id_string', 'timestamp', 'source', 'text',
       'expanded_urls', 'rating_numerator', 'rating_denominator', 'name',
       'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

In [178]:
df_twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2173 entries, 0 to 2355
Data columns (total 13 columns):
tweet_id              2173 non-null int64
tweet_id_string       2173 non-null object
timestamp             2173 non-null object
source                2173 non-null object
text                  2173 non-null object
expanded_urls         2115 non-null object
rating_numerator      2173 non-null int64
rating_denominator    2173 non-null int64
name                  2173 non-null object
doggo                 2173 non-null object
floofer               2173 non-null object
pupper                2173 non-null object
puppo                 2173 non-null object
dtypes: int64(3), object(10)
memory usage: 237.7+ KB


#### Define

##### `Twitter archive enhanced` Table

- Column "timestamp" has "+0000" on every row and is of type String

#### Code

In [179]:
df_twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2173 entries, 0 to 2355
Data columns (total 13 columns):
tweet_id              2173 non-null int64
tweet_id_string       2173 non-null object
timestamp             2173 non-null object
source                2173 non-null object
text                  2173 non-null object
expanded_urls         2115 non-null object
rating_numerator      2173 non-null int64
rating_denominator    2173 non-null int64
name                  2173 non-null object
doggo                 2173 non-null object
floofer               2173 non-null object
pupper                2173 non-null object
puppo                 2173 non-null object
dtypes: int64(3), object(10)
memory usage: 237.7+ KB


In [180]:
df_twitter_clean['timestamp'] = pd.to_datetime(df_twitter_clean['timestamp'])

In [181]:
# adding column 'timestamp_Year' and 'timestamp_month' for later use in visualization
df_twitter_clean['timestamp_Year'] = df_twitter_clean['timestamp'].dt.year
df_twitter_clean['timestamp_month'] = df_twitter_clean['timestamp'].dt.month


In [182]:
# adding column for month name
dictionary_month_name = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8:'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec' }

df_twitter_clean['timestamp_month_name'] = df_twitter_clean['timestamp_month'].apply(lambda x: dictionary_month_name[x] )


#### Test

In [183]:
df_twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2173 entries, 0 to 2355
Data columns (total 16 columns):
tweet_id                2173 non-null int64
tweet_id_string         2173 non-null object
timestamp               2173 non-null datetime64[ns]
source                  2173 non-null object
text                    2173 non-null object
expanded_urls           2115 non-null object
rating_numerator        2173 non-null int64
rating_denominator      2173 non-null int64
name                    2173 non-null object
doggo                   2173 non-null object
floofer                 2173 non-null object
pupper                  2173 non-null object
puppo                   2173 non-null object
timestamp_Year          2173 non-null int64
timestamp_month         2173 non-null int64
timestamp_month_name    2173 non-null object
dtypes: datetime64[ns](1), int64(5), object(10)
memory usage: 288.6+ KB


#### Define

##### `Twitter archive enhanced` Table

- Some observations have listed two stage for a single dog.

#### Code

In [184]:
# removing rows where dog is listed with two stages. 
# check for "doggo" and "floofer"

df_twitter_clean[ (df_twitter_clean.doggo == "doggo") & (df_twitter_clean.puppo == "floofer") ]

,tweet_id,tweet_id_string,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,timestamp_Year,timestamp_month,timestamp_month_name


In [185]:
# check for "doggo" and "puppo"
df_twitter_clean[ (df_twitter_clean.doggo == "doggo") & (df_twitter_clean.puppo == "puppo") ]

,tweet_id,tweet_id_string,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,timestamp_Year,timestamp_month,timestamp_month_name
191,855851453814013952,855851453814013952,2017-04-22 18:31:02,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here's a puppo participating in the #ScienceMarch. Cleverly disguising her own doggo agenda. 13/10 would keep the planet habitable for https://t.co/cMhq16isel,https://twitter.com/dog_rates/status/855851453814013952/photo/1,13,10,None,doggo,None,None,puppo,2017,4,Apr


In [79]:
# check for "doggo" and "pupper"
df_twitter_clean[ (df_twitter_clean.doggo == "doggo") & (df_twitter_clean.puppo == "pupper") ]

,tweet_id,tweet_id_string,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,timestamp_Year,timestamp_month,timestamp_month_name


In [39]:
# check for "floofer" and "pupper"
df_twitter_clean[ (df_twitter_clean.doggo == "floofer") & (df_twitter_clean.puppo == "pupper") ]

,tweet_id,tweet_id_string,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,timestamp_Year,timestamp_month


In [95]:
# check for "floofer" and "puppo"
df_twitter_clean[ (df_twitter_clean.doggo == "floofer") & (df_twitter_clean.puppo == "puppo") ]

,tweet_id,tweet_id_string,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,timestamp_Year,timestamp_month,timestamp_month_name


In [41]:
# check for "puppo" and "pupper"
df_twitter_clean[ (df_twitter_clean.doggo == "puppo") & (df_twitter_clean.puppo == "pupper") ]

,tweet_id,tweet_id_string,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,timestamp_Year,timestamp_month


In [186]:
df_twitter_clean[df_twitter_clean.tweet_id == 855851453814013952]

,tweet_id,tweet_id_string,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,timestamp_Year,timestamp_month,timestamp_month_name
191,855851453814013952,855851453814013952,2017-04-22 18:31:02,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here's a puppo participating in the #ScienceMarch. Cleverly disguising her own doggo agenda. 13/10 would keep the planet habitable for https://t.co/cMhq16isel,https://twitter.com/dog_rates/status/855851453814013952/photo/1,13,10,None,doggo,None,None,puppo,2017,4,Apr


In [187]:
# Thus dropping rows having "tweet_id" = 855851453814013952
df_twitter_clean = df_twitter_clean[df_twitter_clean.tweet_id_string != '855851453814013952']

#### Test

In [188]:
df_twitter_clean[df_twitter_clean.tweet_id == 855851453814013952]

,tweet_id,tweet_id_string,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,timestamp_Year,timestamp_month,timestamp_month_name


In [189]:
df_twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2172 entries, 0 to 2355
Data columns (total 16 columns):
tweet_id                2172 non-null int64
tweet_id_string         2172 non-null object
timestamp               2172 non-null datetime64[ns]
source                  2172 non-null object
text                    2172 non-null object
expanded_urls           2114 non-null object
rating_numerator        2172 non-null int64
rating_denominator      2172 non-null int64
name                    2172 non-null object
doggo                   2172 non-null object
floofer                 2172 non-null object
pupper                  2172 non-null object
puppo                   2172 non-null object
timestamp_Year          2172 non-null int64
timestamp_month         2172 non-null int64
timestamp_month_name    2172 non-null object
dtypes: datetime64[ns](1), int64(5), object(10)
memory usage: 288.5+ KB


In [190]:
df_twitter_clean.doggo.value_counts()

None     2086
doggo    86  
Name: doggo, dtype: int64

In [191]:
df_twitter_clean.floofer.value_counts()

None       2162
floofer    10  
Name: floofer, dtype: int64

In [192]:
df_twitter_clean.pupper.value_counts()

None      1938
pupper    234 
Name: pupper, dtype: int64

In [193]:
df_twitter_clean.puppo.value_counts()

None     2148
puppo    24  
Name: puppo, dtype: int64

#### Define

##### `Twitter archive enhanced` Table

- Actual rating in Column 'rating_numerator' is different from assigned in  column 'text. After decimal,  values are selected instead of the whole number

#### Code

In [194]:
# checking for rating in colum string
df_twitter_clean_decimal_numerator = df_twitter_clean.text.str.extract(r"(\d+\.\d*\/\d+)") 
df_twitter_clean_decimal_numerator = df_twitter_clean_decimal_numerator.dropna()
df_twitter_clean_decimal_numerator 

,0
45,13.5/10
695,9.75/10
763,11.27/10
1689,9.5/10
1712,11.26/10


In [195]:
# comparing with actual values in colum rating_numerator for index. 
list_rating_numerator_whole_number_index = []
for item in df_twitter_clean_decimal_numerator .index:
    list_rating_numerator_whole_number_index.append(item)
for index in list_rating_numerator_whole_number_index:
    print(index, df_twitter.loc[index]['rating_numerator'])

45 5
695 75
763 27
1689 5
1712 26


In [196]:
# creating a list of 'tweet_id"

list_tweet_id = []
for index in list_rating_numerator_whole_number_index:
    list_tweet_id.append(df_twitter_clean.loc[index][0])
    
print(list_tweet_id)


[883482846933004288, 786709082849828864, 778027034220126208, 681340665377193984, 680494726643068929]


In [197]:
# Correcting column rating_numerator
df_twitter_clean.loc[df_twitter.tweet_id == 883482846933004288, 'rating_numerator'] = 13
df_twitter_clean.loc[df_twitter.tweet_id == 832215909146226688, 'rating_numerator'] = 9
df_twitter_clean.loc[df_twitter.tweet_id == 786709082849828864, 'rating_numerator'] = 9
df_twitter_clean.loc[df_twitter.tweet_id == 778027034220126208, 'rating_numerator'] = 11
df_twitter_clean.loc[df_twitter.tweet_id == 681340665377193984, 'rating_numerator'] = 9
df_twitter_clean.loc[df_twitter.tweet_id == 680494726643068929, 'rating_numerator'] = 11


#### Test

In [198]:
# comparing with actual values in colum rating_numerator for index. 
list_rating_numerator_whole_number_index = []
for item in df_twitter_clean_decimal_numerator .index:
    list_rating_numerator_whole_number_index.append(item)
for index in list_rating_numerator_whole_number_index:
    print(index, df_twitter_clean.loc[index]['rating_numerator'])

45 13
695 9
763 11
1689 9
1712 11


#### Define

#### `Image predictions` Table

- Column 'tweet_id' is of type integer. It should be of type string.

#### Code

In [199]:
df_image_clean['tweet_id_string'] = df_image_clean['tweet_id'].astype(str)

In [201]:
df_image_clean.columns

Index(['tweet_id', 'jpg_url', 'img_num', 'p1', 'p1_conf', 'p1_dog', 'p2',
       'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog', 'tweet_id_string'],
      dtype='object')

In [202]:
# Rearranging the column name
df_image_clean = df_image_clean[['tweet_id', 'tweet_id_string', 'jpg_url', 'img_num', 'p1', 'p1_conf', 'p1_dog', 'p2',
       'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog' ]]

#### Test

In [203]:
df_image_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 13 columns):
tweet_id           2075 non-null int64
tweet_id_string    2075 non-null object
jpg_url            2075 non-null object
img_num            2075 non-null int64
p1                 2075 non-null object
p1_conf            2075 non-null float64
p1_dog             2075 non-null bool
p2                 2075 non-null object
p2_conf            2075 non-null float64
p2_dog             2075 non-null bool
p3                 2075 non-null object
p3_conf            2075 non-null float64
p3_dog             2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(5)
memory usage: 168.3+ KB


#### Define

#### `Tweet via  API` Table

- Column 'tweet_id' is of type integer. It should be of type string.

In [204]:
df_tweet_viaAPI_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2338 entries, 0 to 2337
Data columns (total 3 columns):
tweet_id          2338 non-null int64
retweet_count     2338 non-null int64
favorite_count    2338 non-null int64
dtypes: int64(3)
memory usage: 54.9 KB


#### Code

In [205]:
df_tweet_viaAPI_clean['tweet_id_string'] = df_tweet_viaAPI_clean['tweet_id'].astype(str)

In [206]:
df_tweet_viaAPI_clean.columns

Index(['tweet_id', 'retweet_count', 'favorite_count', 'tweet_id_string'], dtype='object')

In [207]:
# Rearranging the column name
df_tweet_viaAPI_clean = df_tweet_viaAPI_clean[['tweet_id', 'tweet_id_string', 'retweet_count', 'favorite_count']]

#### Test

In [208]:
df_tweet_viaAPI_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2338 entries, 0 to 2337
Data columns (total 4 columns):
tweet_id           2338 non-null int64
tweet_id_string    2338 non-null object
retweet_count      2338 non-null int64
favorite_count     2338 non-null int64
dtypes: int64(3), object(1)
memory usage: 73.1+ KB


### Tidiness

#### Define

- Dog stages in separate columns

#### Code

In [209]:
# Creating new column 'stage' and inserting stage type as categorical 
list_condition_stage = [ (df_twitter_clean['doggo'] == 'doggo'),
         (df_twitter_clean['floofer'] == 'floofer'),
         (df_twitter_clean['pupper'] == 'pupper'),
         (df_twitter_clean['puppo'] == 'puppo')]

list_choice_stage_type = ['doggo', 'floofer', 'pupper', 'puppo']

df_twitter_clean['stage'] = np.select(list_condition_stage, list_choice_stage_type, default='unknown_stage')


df_twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2172 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                2172 non-null int64
tweet_id_string         2172 non-null object
timestamp               2172 non-null datetime64[ns]
source                  2172 non-null object
text                    2172 non-null object
expanded_urls           2114 non-null object
rating_numerator        2172 non-null int64
rating_denominator      2172 non-null int64
name                    2172 non-null object
doggo                   2172 non-null object
floofer                 2172 non-null object
pupper                  2172 non-null object
puppo                   2172 non-null object
timestamp_Year          2172 non-null int64
timestamp_month         2172 non-null int64
timestamp_month_name    2172 non-null object
stage                   2172 non-null object
dtypes: datetime64[ns](1), int64(5), object(11)
memory usage: 385.4+ KB


In [210]:
# dropping columns 'doggo', 'floofer', 'pupper', 'puppo'
df_twitter_clean = df_twitter_clean.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis=1)

#### Test

In [211]:
df_twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2172 entries, 0 to 2355
Data columns (total 13 columns):
tweet_id                2172 non-null int64
tweet_id_string         2172 non-null object
timestamp               2172 non-null datetime64[ns]
source                  2172 non-null object
text                    2172 non-null object
expanded_urls           2114 non-null object
rating_numerator        2172 non-null int64
rating_denominator      2172 non-null int64
name                    2172 non-null object
timestamp_Year          2172 non-null int64
timestamp_month         2172 non-null int64
timestamp_month_name    2172 non-null object
stage                   2172 non-null object
dtypes: datetime64[ns](1), int64(5), object(7)
memory usage: 317.6+ KB


In [212]:
df_image_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 13 columns):
tweet_id           2075 non-null int64
tweet_id_string    2075 non-null object
jpg_url            2075 non-null object
img_num            2075 non-null int64
p1                 2075 non-null object
p1_conf            2075 non-null float64
p1_dog             2075 non-null bool
p2                 2075 non-null object
p2_conf            2075 non-null float64
p2_dog             2075 non-null bool
p3                 2075 non-null object
p3_conf            2075 non-null float64
p3_dog             2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(5)
memory usage: 168.3+ KB


In [213]:
df_tweet_viaAPI_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2338 entries, 0 to 2337
Data columns (total 4 columns):
tweet_id           2338 non-null int64
tweet_id_string    2338 non-null object
retweet_count      2338 non-null int64
favorite_count     2338 non-null int64
dtypes: int64(3), object(1)
memory usage: 73.1+ KB


#### Define

- 'text' column in the "Twitter archive " table should split  'tint_url' into a separate column.

#### Code

In [214]:
df_twitter_clean['tiny_url'] = df_twitter_clean.text.str.extract(r"(http[s]:\/\/t\.co\/?[a-zA-Z0-9]{6,})", expand=True)

#### Test

In [215]:
df_twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2172 entries, 0 to 2355
Data columns (total 14 columns):
tweet_id                2172 non-null int64
tweet_id_string         2172 non-null object
timestamp               2172 non-null datetime64[ns]
source                  2172 non-null object
text                    2172 non-null object
expanded_urls           2114 non-null object
rating_numerator        2172 non-null int64
rating_denominator      2172 non-null int64
name                    2172 non-null object
timestamp_Year          2172 non-null int64
timestamp_month         2172 non-null int64
timestamp_month_name    2172 non-null object
stage                   2172 non-null object
tiny_url                2114 non-null object
dtypes: datetime64[ns](1), int64(5), object(8)
memory usage: 334.5+ KB


#### Define

- Merging  Three Dataframes: "df_twitter_clean", "df_tweet_viaAPI_clean", and df_image_clean", to "df_master_twitter".


#### Code

- Merging "df_twitter_clean" and  "df_tweet_viaAPI_clean",  to "df_merge_twitter_viaAPI"

In [216]:
df_merge_twitter_viaAPI = pd.merge(df_twitter_clean, df_tweet_viaAPI_clean, on='tweet_id_string', how='inner')

In [120]:
df_merge_twitter_viaAPI.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2335 entries, 0 to 2334
Data columns (total 17 columns):
tweet_id_x              2335 non-null int64
tweet_id_string         2335 non-null object
timestamp               2335 non-null datetime64[ns]
source                  2335 non-null object
text                    2335 non-null object
expanded_urls           2276 non-null object
rating_numerator        2335 non-null int64
rating_denominator      2335 non-null int64
name                    2335 non-null object
timestamp_Year          2335 non-null int64
timestamp_month         2335 non-null int64
timestamp_month_name    2335 non-null object
stage                   2335 non-null object
tiny_url                2203 non-null object
tweet_id_y              2335 non-null int64
retweet_count           2335 non-null int64
favorite_count          2335 non-null int64
dtypes: datetime64[ns](1), int64(8), object(8)
memory usage: 328.4+ KB


- Merging "df_merge_twitter_viaAPI" and  "df_image_clean", to "df_master_twitter"

In [217]:
df_master_twitter = pd.merge(df_merge_twitter_viaAPI, df_image_clean, on='tweet_id_string', how='inner')

In [218]:
df_master_twitter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1987 entries, 0 to 1986
Data columns (total 29 columns):
tweet_id_x              1987 non-null int64
tweet_id_string         1987 non-null object
timestamp               1987 non-null datetime64[ns]
source                  1987 non-null object
text                    1987 non-null object
expanded_urls           1987 non-null object
rating_numerator        1987 non-null int64
rating_denominator      1987 non-null int64
name                    1987 non-null object
timestamp_Year          1987 non-null int64
timestamp_month         1987 non-null int64
timestamp_month_name    1987 non-null object
stage                   1987 non-null object
tiny_url                1987 non-null object
tweet_id_y              1987 non-null int64
retweet_count           1987 non-null int64
favorite_count          1987 non-null int64
tweet_id                1987 non-null int64
jpg_url                 1987 non-null object
img_num                 1987 non-null i

In [219]:
# dropping column 'tweet_id_x','tweet_id_y', 'tweet_id'
df_master_twitter = df_master_twitter.drop(['tweet_id_x','tweet_id_y', 'tweet_id'], axis=1)

#### Test

In [220]:
df_master_twitter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1987 entries, 0 to 1986
Data columns (total 26 columns):
tweet_id_string         1987 non-null object
timestamp               1987 non-null datetime64[ns]
source                  1987 non-null object
text                    1987 non-null object
expanded_urls           1987 non-null object
rating_numerator        1987 non-null int64
rating_denominator      1987 non-null int64
name                    1987 non-null object
timestamp_Year          1987 non-null int64
timestamp_month         1987 non-null int64
timestamp_month_name    1987 non-null object
stage                   1987 non-null object
tiny_url                1987 non-null object
retweet_count           1987 non-null int64
favorite_count          1987 non-null int64
jpg_url                 1987 non-null object
img_num                 1987 non-null int64
p1                      1987 non-null object
p1_conf                 1987 non-null float64
p1_dog                  1987 non-nul

In [221]:
# save to CSV
df_master_twitter.to_csv('twitter_archive_master.csv', index=False, encoding='utf-8')

In [222]:
df_master_twitter = pd.read_csv('twitter_archive_master.csv')
df_master_twitter.head(2)

,tweet_id_string,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,timestamp_Year,timestamp_month,timestamp_month_name,stage,tiny_url,retweet_count,favorite_count,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,892420643555336193,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,2017,8,Aug,unknown_stage,https://t.co/MgUWQ76dJU,8443,38416,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,1,orange,0.097049,False,bagel,0.085851,False,banana,0.076110,False
1,892177421306343426,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,2017,8,Aug,unknown_stage,https://t.co/0Xxu71qeIV,6220,32908,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True


### Reference

- https://stackoverflow.com/questions/23667369/drop-all-duplicate-rows-in-python-pandas
- http://pandas.pydata.org/pandas-docs/stable/10min.html#selection-by-label
- https://stackoverflow.com/questions/19124601/is-there-a-way-to-pretty-print-the-entire-pandas-series-dataframe
- https://stackoverflow.com/questions/12329853/how-to-rearrange-pandas-column-sequence/23741704
- https://www.programcreek.com/python/example/41297/numpy.select
- https://docs.scipy.org/doc/numpy/reference/generated/numpy.select.html
- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html